<a href="https://colab.research.google.com/github/MohammedFarzin/Speech-Emotion-Recognition-using-TIM-Net/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Importing libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Speech Emotion Recognition/TIM-NET

/content/drive/MyDrive/Colab Notebooks/Speech Emotion Recognition/TIM-NET


In [4]:
import numpy as np
import tensorflow.keras.backend as K
import os
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Layer, Dense, Input
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from Common_Model import Common_Model
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import datetime
import pandas as pd
from TIMNET import TIMNET

In [8]:
# Smoothing labels
def smooth_lables(labels, factor=0.1):
    labels *= (1-factor)
    labels += (factor/labels.shape[1])
    return labels

# softmax
def softmax(x, axis=-1):
  ex = K.ex(x - K.max(x, axis=axis, keepdims=True))
  return ex/K.sum(ex, axis=axis, keepdims=True)

In [9]:
class WeightLayer(Layer):
  def __init__(self, **kwargs):
    super(WeightLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    self.kernel = self.add_weight(name='kernel', shape=(input_shape[1], 1), initializer='uniform', trainable=True)
    super(WeightLayer, self).build(input_shape)

  def call(self, x):
    tempx = tf.transpose(x, [0, 2, 1])
    x = K.dot(tempx, self.kernel)
    x = tf.squeeze(x, axis=-1)
    return x

  def compute_output_shape(self, input_shape):
    return (input_shape[0], input_shape[2])

In [ ]:
class TIMNET_Model(Common_Model):
  def __init__(self, args, input_shape, class_label, **params):
    super(TIMNET_Model, self).__init__(**params)
    self.args = args
    self.data_shape = input_shape
    self.num_classes = len(class_label)
    self.class_label = class_label
    self.matrix = []
    self.eva_matrix = []
    self.acc = 0
    print('TIMNET MODEL SHAPE: ', input_shape)

  def create_model(self):
    self.inputs = Input(shape = (self.data_shape[0], self.data_shape[1]))
    self.multi_decision = TIMNET(nb_filters=self.args.filter_size, kernel_size=self.args.kernel_size, nb_stack=self.args.stack_size, dilation=self.args.dilation_size,
                                 activation=self.args.activation, dropout_rate=self.args.dropout_rate, return_sequence=True, name='TIMNET')(self.inputs)

